In [ ]:
import findspark
findspark.init()
findspark.find()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
            SparkSession
                .builder
                .appName("SparkSQLApp")
                .master("local[4]")
    
                .config("spark.dynamicAllocation.enabled", "false")
                .config("spark.sql.adaptive.enabled", "false")
    
                .getOrCreate()
        )

sc = spark.sparkContext

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/02 12:06:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/02 12:06:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/02 12:06:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/05/02 12:06:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
from IPython.display import *
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Read Yellow Taxis data

In [4]:
# Create schema for Yellow Taxi Data

taxiSchema = (
                    StructType
                    ([ 
                        StructField("VendorId"               , IntegerType()   , True),
                        StructField("lpep_pickup_datetime"   , TimestampType() , True),
                        StructField("lpep_dropoff_datetime"  , TimestampType() , True),                            
                        StructField("passenger_count"        , DoubleType()    , True),
                        StructField("trip_distance"          , DoubleType()    , True),
                        StructField("RatecodeID"             , DoubleType()    , True),                            
                        StructField("store_and_fwd_flag"     , StringType()    , True),
                        StructField("PULocationID"           , IntegerType()   , True),
                        StructField("DOLocationID"           , IntegerType()   , True),                            
                        StructField("payment_type"           , IntegerType()   , True),                            
                        StructField("fare_amount"            , DoubleType()    , True),
                        StructField("extra"                  , DoubleType()    , True),
                        StructField("mta_tax"                , DoubleType()    , True),
                        StructField("tip_amount"             , DoubleType()    , True),
                        StructField("tolls_amount"           , DoubleType()    , True),
                        StructField("improvement_surcharge"  , DoubleType()    , True),
                        StructField("total_amount"           , DoubleType()    , True),
                        StructField("congestion_surcharge"   , DoubleType()    , True),
                        StructField("airport_fee"            , DoubleType()    , True)
                    ])
               )

In [5]:
# Read YellowTaxis CSV file

yellowTaxiDF = (
                  spark
                    .read
                    .option("header", "true")
    
                    .schema(taxiSchema)
    
                    .csv("./../files/input/YellowTaxis_202210.csv")
               )

yellowTaxiDF.printSchema()

root
 |-- VendorId: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



### Create a SQL View based on Python DataFrame

In [6]:
yellowTaxiDF.createOrReplaceTempView("YellowTaxis")

### Run SQL query on View

In [7]:
outputDF = spark.sql(
                        "SELECT * FROM YellowTaxis WHERE PULocationID = 171"
                    )
    
outputDF.show()

23/05/02 12:07:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee
 Schema: VendorId, lpep_pickup_datetime, lpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee
Expected: lpep_pickup_datetime but found: tpep_pickup_datetime
CSV file: file:///Users/mac/IdeaProjects/TestSpark3/files/input/YellowTaxis_202210.csv


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorId|lpep_pickup_datetime|lpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-10-01 11:17:23|  2022-10-01 12:08:50|            1.0|          9.4|      99.0|                 N|         171|         263|           1|       35.2|  0.0|    0.5|       0.

### Read Green Taxis data, and create Temp View

In [8]:
# Read GreenTaxis TSV file

greenTaxiDF = (
                  spark
                    .read                     
                    .option("header", "true")    
                    .option("delimiter", "\t")    
                    .csv("./../files/input/GreenTaxis_202210.csv")
              )

greenTaxiDF.createOrReplaceTempView("GreenTaxis")

### Write SQL query to merge Yellow and Green Taxis data

In [9]:
spark.sql("""

SELECT 'Yellow'                   AS TaxiType

      , lpep_pickup_datetime      AS PickupTime
      , lpep_dropoff_datetime     AS DropTime
      , PULocationID              AS PickupLocationId
      , DOLocationID              AS DropLocationId      
FROM YellowTaxis

UNION ALL

SELECT 'Green'                    AS TaxiType

      , lpep_pickup_datetime      AS PickupTime
      , lpep_dropoff_datetime     AS DropTime
      , PULocationID              AS PickupLocationId
      , DOLocationID              AS DropLocationId 
FROM GreenTaxis

""").show()

+--------+-------------------+-------------------+----------------+--------------+
|TaxiType|         PickupTime|           DropTime|PickupLocationId|DropLocationId|
+--------+-------------------+-------------------+----------------+--------------+
|  Yellow|2022-10-01 03:03:41|2022-10-01 03:18:39|             249|           107|
|  Yellow|2022-10-01 03:14:30|2022-10-01 03:19:48|             151|           238|
|  Yellow|2022-10-01 03:27:13|2022-10-01 03:37:41|             238|           166|
|  Yellow|2022-10-01 03:32:53|2022-10-01 03:38:55|             142|           239|
|  Yellow|2022-10-01 03:44:55|2022-10-01 03:50:21|             238|           166|
|  Yellow|2022-10-01 03:22:52|2022-10-01 03:52:14|             186|            41|
|  Yellow|2022-10-01 03:33:19|2022-10-01 03:44:51|             162|           145|
|  Yellow|2022-10-01 03:02:42|2022-10-01 03:50:01|             100|            22|
|  Yellow|2022-10-01 03:06:35|2022-10-01 03:24:38|             138|           112|
|  Y

23/05/02 12:07:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: tpep_pickup_datetime, tpep_dropoff_datetime, PULocationID, DOLocationID
 Schema: lpep_pickup_datetime, lpep_dropoff_datetime, PULocationID, DOLocationID
Expected: lpep_pickup_datetime but found: tpep_pickup_datetime
CSV file: file:///Users/mac/IdeaProjects/TestSpark3/files/input/YellowTaxis_202210.csv
